In [10]:
# Importamos bibliotecas
import numpy as np
import pandas as pd
from datetime import datetime
import re
from sklearn.ensemble import IsolationForest

# Establecemos las columnas
colz=["IP","timestamp","status code","bytes sent","user agent"]

li=[]
c=0

# Abrimos el archivo
with open("../logs/access_ssl_20230404.log") as f:
    while(True):
        if(c==2000000):
            break
        if(c%1000000==0):
            print(c) # Esto nose, yo lo borraria
        try:
            x=f.readline()
            if x==None:
                break
            log=x.split(" ")
            if(len(log)<12):
                continue
            #print([log[0],log[3].strip('['),log[5].strip('"'),log[7].strip('"'),int(log[8]),int(log[9]),' '.join(log[11:-1]).strip('"')])
            li.append([log[0],log[3].strip('['),int(log[8]),int(log[9]),' '.join(log[11:-1]).strip('"')])
        except Exception as e:
            pass
        c+=1

# basic feature engineering
logs_df=pd.DataFrame(li,columns=colz)
logs_df["timestamp"]=pd.to_datetime(logs_df["timestamp"],format='%d/%b/%Y:%H:%M:%S')


0
1000000


In [2]:
from ua_parser import user_agent_parser

def parse_navegador(ua):
    parsed = user_agent_parser.Parse(ua)
    navegador = parsed["user_agent"]["family"]
    return pd.Series([navegador])

logs_df[["navegador"]] = logs_df["user agent"].apply(parse_navegador)

In [2]:
logs_df

NameError: name 'logs_df' is not defined

In [1]:
# Nos quedamos solamente con las variables que nos interesan
navegadores = logs_df[["navegador"]]

# Eliminamos duplicados
navegadores = navegadores.drop_duplicates().reset_index(drop=True)

# Añadimos el ID
navegadores.insert(0, "id_navegador", range(1, len(navegadores) + 1))

NameError: name 'logs_df' is not defined

In [ ]:
# Vemos la tabla resultante
navegadores

In [ ]:
# Actualizamos la dimension Navegadores en el DW
from Modulos.update_dimensions_table import actualizarTablaDimension
navegadores = actualizarTablaDimension(engine, "navegadores", navegadores, pk="id_navegador")

In [4]:
import numpy as np
import pandas as pd
from datetime import datetime
import re
from ua_parser import user_agent_parser
from sqlalchemy import create_engine
from Modulos.update_dimensions_table import actualizarTablaDimension

In [5]:
def process_log_file(filepath, chunk_size=100000):
    colz = ["IP", "timestamp", "status code", "bytes sent", "user agent"]
    all_chunks = []
    
    with open("logs/access_ssl_20230404.log", 'r') as f:
        lines_processed = 0
        while True:
            li = []
            # Procesar chunk
            for _ in range(chunk_size):
                try:
                    x = f.readline()
                    if not x:  # EOF
                        break
                    log = x.split(" ")
                    if len(log) < 12:
                        continue
                    li.append([log[0], log[3].strip('['), int(log[8]), int(log[9]), ' '.join(log[11:-1]).strip('"')])
                except Exception as e:
                    continue
            
            if not li:  # No más datos
                break
                
            # Crear DataFrame del chunk
            chunk_df = pd.DataFrame(li, columns=colz)
            chunk_df["timestamp"] = pd.to_datetime(chunk_df["timestamp"], format='%d/%b/%Y:%H:%M:%S')
            all_chunks.append(chunk_df)
            
            lines_processed += len(li)
            print(f"Procesadas {lines_processed} líneas...")
    
    # Combinar todos los chunks
    if all_chunks:
        logs_df = pd.concat(all_chunks, ignore_index=True)
        return logs_df
    else:
        return pd.DataFrame(columns=colz)

# Uso
filepath = "logs/access_ssl_20230404.log"
print("Procesando archivo de logs...")
logs_df = process_log_file(filepath, chunk_size=1000000)
print("Procesamiento completado")
print(logs_df.info())

Procesando archivo de logs...
Procesadas 999958 líneas...
Procesadas 1999910 líneas...
Procesadas 2539378 líneas...
Procesamiento completado
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2539378 entries, 0 to 2539377
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   IP           object        
 1   timestamp    datetime64[ns]
 2   status code  int64         
 3   bytes sent   int64         
 4   user agent   object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 96.9+ MB
None


In [6]:
# -----------------------------------------------------------
# Versión OPTIMIZADA - hasta 100x más rápida
# -----------------------------------------------------------

# Primero obtener user agents únicos
user_agents_unicos = logs_df["user agent"].unique()
print(f"User agents únicos: {len(user_agents_unicos)} de {len(logs_df)}")

# Crear un diccionario cache para no parsear duplicados
navegador_cache = {}

def parse_navegador_rapido(ua):
    if ua not in navegador_cache:
        parsed = user_agent_parser.Parse(ua)
        navegador_cache[ua] = parsed["user_agent"]["family"]
    return navegador_cache[ua]

# Aplicar solo a los únicos primero
print("Parseando user agents únicos...")
navegadores_unicos = [parse_navegador_rapido(ua) for ua in user_agents_unicos]

# Crear el mapeo completo
navegador_por_ua = dict(zip(user_agents_unicos, navegadores_unicos))

# Aplicar el mapeo a todo el DataFrame (instantáneo)
logs_df["navegador"] = logs_df["user agent"].map(navegador_por_ua)

# Crear la dimensión
navegadores = pd.DataFrame({
    'navegador': navegadores_unicos
}).drop_duplicates().reset_index(drop=True)

navegadores.insert(0, "id_navegador", range(1, len(navegadores) + 1))

print(navegadores.head())

User agents únicos: 9704 de 2539378
Parseando user agents únicos...
   id_navegador              navegador
0             1                 Chrome
1             2                  Other
2             3              WordPress
3             4  Chrome Mobile WebView
4             5              Googlebot


In [7]:
navegadores

,id_navegador,navegador
0,1,Chrome
1,2,Other
2,3,WordPress
3,4,Chrome Mobile WebView
4,5,Googlebot
...,...,...
117,118,bitlybot
118,119,InternetArchive
119,120,Scribdbot
120,121,t3versionsBot


In [ ]:
# -----------------------------------------------------------
# Versión OPTIMIZADA de navegador dispositivo
# -----------------------------------------------------------

from ua_parser import user_agent_parser

unique_uas = logs_df["user agent"].unique()

# Parsear únicos
def parse_ua(ua):
    parsed = user_agent_parser.Parse(ua)
    return (
        parsed["user_agent"]["family"] or "desconocido",
        parsed["os"]["family"] or "desconocido",
        parsed["device"]["brand"] or "desconocido",
        parsed["device"]["model"] or "desconocido"
    )

parsed_unique = pd.DataFrame(
    [parse_ua(ua) for ua in unique_uas],
    columns=["navegador", "so", "marca", "modelo"],
    index=unique_uas
)

# Mapear de vuelta a logs_df
logs_df = logs_df.join(parsed_unique, on="user agent")
print(logs_df)